In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import pyspark.sql.functions as F
spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()

In [2]:
# Read lines from the text file
answers_sdf = spark.read.load('sx-stackoverflow-a2q.txt', format="text")
comments_answers_sdf = spark.read.load('sx-stackoverflow-c2a.txt', format="text")
comments_questions_sdf = spark.read.load('sx-stackoverflow-c2q.txt', format="text")

## Step 2.2

In [3]:
answers_sdf.count()

17823525

In [4]:
answers_sdf.show(10)

+----------------+
|           value|
+----------------+
|  9 8 1217567877|
|  1 1 1217573801|
| 13 1 1217606247|
| 17 1 1217617639|
| 48 2 1217618182|
| 17 1 1217618239|
| 19 9 1217618357|
|13 23 1217618560|
|13 11 1217618799|
|23 23 1217619360|
+----------------+
only showing top 10 rows



In [5]:
answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [6]:
comments_answers_sdf.count()

25405374

In [7]:
comments_answers_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|     1 91 1220713630|
|     3 91 1220713792|
|  380 350 1220715736|
|4642 2257 1220734307|
|4642 1324220 1220...|
|2495 4285 1220736640|
|4642 4893 1220737355|
|2515 4903 1220738560|
|2515 4893 1220739071|
|  199 199 1220741079|
+--------------------+
only showing top 10 rows



In [8]:
comments_answers_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [9]:
comments_questions_sdf.count()

20268151

In [10]:
comments_questions_sdf.show(10)

+--------------------+
|               value|
+--------------------+
|4550 4550 1220729190|
|  242 184 1220733503|
|4213 4946 1220768149|
|    91 91 1220768295|
|2658 1874 1220771891|
|4035 1874 1220773037|
|2257 4489 1220802041|
|  577 577 1220834891|
|4489 4489 1220853536|
| 828 2783 1220854143|
+--------------------+
only showing top 10 rows



In [11]:
comments_questions_sdf.printSchema()

root
 |-- value: string (nullable = true)



In [12]:
answers_sdf.createOrReplaceTempView('answers_sdf_view')
answers_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , CAST(split(value, " ")[2] AS string) AS from_unixtime,"answer" AS edge_type FROM answers_sdf_view')

comments_answers_sdf.createOrReplaceTempView('comments_answers_sdf_view')
comments_answers_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , CAST(split(value, " ")[2] AS string) AS from_unixtime,"comment-on-answer" AS edge_type FROM comments_answers_sdf_view')

comments_questions_sdf.createOrReplaceTempView('comments_questions_sdf_view')
comments_questions_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , CAST(split(value, " ")[2] AS string) AS from_unixtime,"comment-on-question" AS edge_type FROM comments_questions_sdf_view')

In [13]:
graph_sdf=spark.sql('SELECT CAST(split(value, " ")[0] AS integer) AS from_node ,CAST(split(value, " ")[1] AS integer) AS to_node , CAST(split(value, " ")[2] AS string) AS from_unixtime,"answer" AS edge_type FROM answers_sdf_view')
graph_sdf.unionAll(comments_answers_sdf).unionAll(comments_questions_sdf)

DataFrame[from_node: int, to_node: int, from_unixtime: string, edge_type: string]

## Step 2.3

In [14]:
answers_sdf.count()

17823525

In [15]:
answers_sdf.show(5)

+---------+-------+-------------+---------+
|from_node|to_node|from_unixtime|edge_type|
+---------+-------+-------------+---------+
|        9|      8|   1217567877|   answer|
|        1|      1|   1217573801|   answer|
|       13|      1|   1217606247|   answer|
|       17|      1|   1217617639|   answer|
|       48|      2|   1217618182|   answer|
+---------+-------+-------------+---------+
only showing top 5 rows



In [16]:
answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- from_unixtime: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [17]:
comments_answers_sdf.count()

25405374

In [18]:
comments_answers_sdf.show(5)

+---------+-------+-------------+-----------------+
|from_node|to_node|from_unixtime|        edge_type|
+---------+-------+-------------+-----------------+
|        1|     91|   1220713630|comment-on-answer|
|        3|     91|   1220713792|comment-on-answer|
|      380|    350|   1220715736|comment-on-answer|
|     4642|   2257|   1220734307|comment-on-answer|
|     4642|1324220|   1220735746|comment-on-answer|
+---------+-------+-------------+-----------------+
only showing top 5 rows



In [19]:
comments_answers_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- from_unixtime: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [20]:
comments_questions_sdf.count()

20268151

In [21]:
comments_questions_sdf.show(5)

+---------+-------+-------------+-------------------+
|from_node|to_node|from_unixtime|          edge_type|
+---------+-------+-------------+-------------------+
|     4550|   4550|   1220729190|comment-on-question|
|      242|    184|   1220733503|comment-on-question|
|     4213|   4946|   1220768149|comment-on-question|
|       91|     91|   1220768295|comment-on-question|
|     2658|   1874|   1220771891|comment-on-question|
+---------+-------+-------------+-------------------+
only showing top 5 rows



In [22]:
comments_questions_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- from_unixtime: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [23]:
graph_sdf.count()

17823525

In [24]:
graph_sdf.show(5)

+---------+-------+-------------+---------+
|from_node|to_node|from_unixtime|edge_type|
+---------+-------+-------------+---------+
|        9|      8|   1217567877|   answer|
|        1|      1|   1217573801|   answer|
|       13|      1|   1217606247|   answer|
|       17|      1|   1217617639|   answer|
|       48|      2|   1217618182|   answer|
+---------+-------+-------------+---------+
only showing top 5 rows



In [27]:
graph_sdf.printSchema()

root
 |-- from_node: integer (nullable = true)
 |-- to_node: integer (nullable = true)
 |-- from_unixtime: string (nullable = true)
 |-- edge_type: string (nullable = false)



In [68]:
graph_sdf.createOrReplaceTempView('graph_sdf_view')
X=spark.sql('SELECT to_node, COUNT(*) AS indegree FROM graph_sdf_view GROUP BY to_node ORDER BY COUNT(*) DESC LIMIT 5')
X.show()

+-------+--------+
|to_node|indegree|
+-------+--------+
|   4653|    5453|
|  39677|    4971|
|  34537|    4391|
| 179736|    3716|
|  84201|    3710|
+-------+--------+



In [69]:
graph_sdf.createOrReplaceTempView('graph_sdf_view')
Y=spark.sql('SELECT from_node, COUNT(*) AS out FROM graph_sdf_view GROUP BY from_node ORDER BY COUNT(*) DESC LIMIT 5')
Y.show()

+-------+--------+
|to_node|indegree|
+-------+--------+
|   4653|    5453|
|  39677|    4971|
|  34537|    4391|
| 179736|    3716|
|  84201|    3710|
+-------+--------+

